In [1]:
import faiss
import numpy as np

import torch

In [2]:
ngpus = faiss.get_num_gpus()

In [3]:
d = 512                         # dimension
nb = 100000                     # database size
nq = 128                      # nb of queries
np.random.seed(1234)            # make reproducible
# xb = np.random.random((nb, d)).astype('float32')
# xb[:, 0] += np.arange(nb) / 1000.
# xq = np.random.random((nq, d)).astype('float32')
# xq[:, 0] += np.arange(nq) / 1000.

In [9]:
xb = np.random.random((nb*10, d)).astype('float32')
xq = xb[-nq*100:].copy()

In [10]:
cpu_index = faiss.IndexFlatL2(d)

In [11]:
gpu_index = faiss.index_cpu_to_all_gpus(  # build the index
    cpu_index
)

In [13]:
gpu_index.add(xb)              # add vectors to the index
print(gpu_index.ntotal//1e6, 'M')

2.0 M


In [14]:
%%timeit
k = 4                          # we want to see 4 nearest neighbors
D, I = gpu_index.search(xq, k) # actual search

429 ms ± 14.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [17]:
k = 4                          # we want to see 4 nearest neighbors
D, I = gpu_index.search(xq, k) # actual search
print(D[0], I[0])

[1.5258789e-04 1.5258789e-04 6.8656708e+01 6.8656708e+01] [1987200  987200   55528 1055528]


In [16]:
D.shape, I.shape

((12800, 4), (12800, 4))

In [22]:
%%timeit
torch.nn.Softmax(dim=1)(1/torch.tensor(np.array([D[0]]), dtype=torch.float32))

26.9 µs ± 2.32 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
